In [1]:
import os

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from joblib import dump, load
import warnings

from sklearn import metrics

from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis
from snorkel.labeling.model import LabelModel, MajorityLabelVoter

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [2]:
n_folds = 9
output_sfx = '8.26.21-0.9-10.20.21'
output_dir = '../input/%dfold/%s' % (n_folds, '%s')
train_all_fn = output_dir % ('train-all-%s.pickle' % output_sfx)
train_fn = output_dir % ('train-f%s-%s.pickle' % ('%d', output_sfx))
valid_fn = output_dir % ('valid-f%s-%s.pickle' % ('%d', output_sfx))
test_fn = output_dir % ('test-%s.pickle' % output_sfx)

In [3]:
def recall(prob, true, text='', disp=True):
    tmp_zip = zip(true, prob)
    recall = [1 if y == 1 and np.argmax(p) == 1
              else 0 for y, p in tmp_zip]
    tmp_zip = zip(true, prob)
    recall_not = [1 if y != 1 and np.argmax(p) != 1
                  else 0 for y, p in tmp_zip]
    tmp_zip = zip(true, prob)
    recall_all = [1 if (y == 1 and np.argmax(p) == 1) 
                  or (y != 1 and np.argmax(p) != 1) 
                  else 0 for y, p in tmp_zip]
    num_true = np.sum([1 if i == 1 else 0 for i in true])
    r_1 = np.sum(recall) / num_true
    r0_1 = np.sum(recall_not) / (len(true) - num_true)
    rall_1 = np.sum(recall_all) / len(true)
    if disp:
        print('%s r %.3f r0 %.3f rall %.3f ravg %.3f' % (text, r_1, r0_1, rall_1, (r_1 + r0_1)/2))
    return r_1, r0_1, rall_1

## Import data

In [4]:
trains = []
valids = []
for n in tqdm(range(n_folds)):
    trains.append(pd.read_pickle(train_fn % n))
    valids.append(pd.read_pickle(valid_fn % n))

train_all = pd.read_pickle(train_all_fn)
test = pd.read_pickle(test_fn)

  0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
X_train = train_all['Cleaned Abstract']
y_train = pd.DataFrame([1 if i == 1 else -1 for i in train_all.Category])

X_test = test['Cleaned Abstract']
y_test = pd.DataFrame([1 if i == 1 else -1 for i in test.Category])

k = 0
X_valid = valids[k]['Cleaned Abstract']
y_valid = pd.DataFrame([1 if i == 1 else -1 for i in valids[k].Category])

## LF labels

In [6]:
pos = 1
neg = 0
abst = -1

@labeling_function()
def ai(x):
    return pos if "artificial intelligence" in x['Cleaned Abstract'] else abst

@labeling_function()
def nn(x):
    return pos if "neural network" in x['Cleaned Abstract'] else abst

@labeling_function()
def ml(x):
    return pos if "machine learning" in x['Cleaned Abstract'] else abst

@labeling_function()
def dl(x):
    return pos if "deep learning" in x['Cleaned Abstract'] else abst

@labeling_function()
def genetic(x):
    return pos if "genetic" in x['Cleaned Abstract'] else abst

@labeling_function()
def cnn(x):
    return pos if "convolutional neural network" in x['Cleaned Abstract'] else abst

@labeling_function()
def genomic(x):
    return pos if "genomic" in x['Cleaned Abstract'] else abst

@labeling_function()
def clingen(x):
    return pos if "clinical genetics" in x['Cleaned Abstract'] else abst

@labeling_function()
def medgen(x):
    return pos if "medical genetics" in x['Cleaned Abstract'] else abst

@labeling_function()
def medgenom(x):
    return pos if "medical genomics" in x['Cleaned Abstract'] else abst

@labeling_function()
def geneticist(x):
    return pos if "geneticist" in x['Cleaned Abstract'] else abst

@labeling_function()
def genommed(x):
    return pos if "genomic medicine" in x['Cleaned Abstract'] else abst

@labeling_function()
def pm(x):
    return pos if "precision medicine" in x['Cleaned Abstract'] else abst

@labeling_function()
def gene_noml(x):
    return neg if "gene" in x['Cleaned Abstract'] and 'machine learning' not in x['Cleaned Abstract'] and 'deep learning' not in x['Cleaned Abstract'] and 'artificial intelligence' not in x['Cleaned Abstract'] else abst

@labeling_function()
def ml_nogene(x):
    return neg if "gene" not in x['Cleaned Abstract'] and 'machine learning' in x['Cleaned Abstract'] and 'deep learning' in x['Cleaned Abstract'] and 'artificial intelligence' in x['Cleaned Abstract'] else abst

lfs = [ai, nn, ml, dl, genetic, cnn, genomic, clingen, medgen, medgenom, geneticist, genommed, pm]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train_all)
L_test = applier.apply(df=test)
print(L_train.shape, L_test.shape)

lfs_neg = [ai, nn, ml, dl, genetic, cnn, genomic, clingen, medgen, medgenom, geneticist, genommed, pm, gene_noml, ml_nogene]
applier = PandasLFApplier(lfs=lfs_neg)
L_train_neg = applier.apply(df=train_all)
L_test_neg = applier.apply(df=test)
print(L_train_neg.shape, L_test_neg.shape)

100%|█████████████████████████████████████████████████████████████████| 1370/1370 [00:00<00:00, 12104.80it/s]


(12326, 13) (1370, 13)


100%|██████████████████████████████████████████████████████████████████| 1370/1370 [00:00<00:00, 6123.01it/s]

(12326, 15) (1370, 15)


In [7]:
L_train_abs = L_train_neg.copy()
L_test_abs = L_test_neg.copy()
n_lfs = len(lfs_neg)

for n in range(L_train_abs.shape[0]):
    if L_train_abs[n].sum() == -n_lfs:
        L_train_abs[n] = np.zeros(n_lfs)

for n in range(L_test_abs.shape[0]):
    if L_test_abs[n].sum() == -n_lfs:
        L_test_abs[n] = np.zeros(n_lfs)

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    print(LFAnalysis(L=L_train, lfs=lfs).lf_summary(y_train.to_numpy()))

             j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
ai           0      [1]  0.097923  0.072286        0.0        6          0   
nn           1      [1]  0.375710  0.330034        0.0       24          0   
ml           2      [1]  0.153334  0.129239        0.0       10          0   
dl           3      [1]  0.609038  0.323868        0.0       47          0   
genetic      4      [1]  0.030261  0.025637        0.0       32          0   
cnn          5      [1]  0.226675  0.226675        0.0       13          0   
genomic      6      [1]  0.025231  0.022067        0.0        3          0   
clingen      7      [1]  0.000325  0.000325        0.0        2          0   
medgen       8      [1]  0.000081  0.000081        0.0        0          0   
medgenom     9       []  0.000000  0.000000        0.0        0          0   
geneticist  10      [1]  0.000568  0.000568        0.0        5          0   
genommed    11      [1]  0.000325  0.000325        0.0        0 

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    print(LFAnalysis(L=L_train_neg, lfs=lfs_neg).lf_summary(np.array([1 if i == 1 else 0 for i in train_all.Category])))

             j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
ai           0      [1]  0.097923  0.072286   0.013062        6       1201   
nn           1      [1]  0.375710  0.346503   0.044297       24       4607   
ml           2      [1]  0.153334  0.129239   0.013062       10       1880   
dl           3      [1]  0.609038  0.323868   0.013062       47       7460   
genetic      4      [1]  0.030261  0.030261   0.008356       32        341   
cnn          5      [1]  0.226675  0.226675   0.022878       13       2781   
genomic      6      [1]  0.025231  0.023528   0.004138        3        308   
clingen      7      [1]  0.000325  0.000325   0.000162        2          2   
medgen       8      [1]  0.000081  0.000081   0.000081        0          1   
medgenom     9       []  0.000000  0.000000   0.000000        0          0   
geneticist  10      [1]  0.000568  0.000568   0.000243        5          2   
genommed    11      [1]  0.000325  0.000325   0.000000        0 

In [10]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    print(LFAnalysis(L=L_train_abs, lfs=lfs_neg).lf_summary(np.array([1 if i == 1 else 0 for i in train_all.Category])))

             j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
ai           0   [0, 1]  0.228379  0.202742   0.013062     1593       1222   
nn           1   [0, 1]  0.506166  0.476959   0.044297     1611       4628   
ml           2   [0, 1]  0.283790  0.259695   0.013062     1597       1901   
dl           3   [0, 1]  0.739494  0.454324   0.013062     1634       7481   
genetic      4   [0, 1]  0.160717  0.160717   0.008356     1619        362   
cnn          5   [0, 1]  0.357131  0.357131   0.022878     1600       2802   
genomic      6   [0, 1]  0.155687  0.153983   0.004138     1590        329   
clingen      7   [0, 1]  0.130780  0.130780   0.000162     1589         23   
medgen       8   [0, 1]  0.130537  0.130537   0.000081     1587         22   
medgenom     9      [0]  0.130456  0.130456   0.000000     1587         21   
geneticist  10   [0, 1]  0.131024  0.131024   0.000243     1592         23   
genommed    11   [0, 1]  0.130780  0.130780   0.000000     1587 

In [11]:
for k in range(L_train_neg.shape[1]):
    correct = np.sum([1 if L_train_neg[n, k] == 1 and y_train.to_numpy()[n] == 1 else 0 for n in range(L_train_neg.shape[0])])
    total = np.sum(train_all.Category == 1)
    print('lf %d category 1: %d correct out of %d, percent %.2f' % (k, correct, total, correct/total))

lf 0 category 1: 6 correct out of 115, percent 0.05
lf 1 category 1: 24 correct out of 115, percent 0.21
lf 2 category 1: 10 correct out of 115, percent 0.09
lf 3 category 1: 47 correct out of 115, percent 0.41
lf 4 category 1: 32 correct out of 115, percent 0.28
lf 5 category 1: 13 correct out of 115, percent 0.11
lf 6 category 1: 3 correct out of 115, percent 0.03
lf 7 category 1: 2 correct out of 115, percent 0.02
lf 8 category 1: 0 correct out of 115, percent 0.00
lf 9 category 1: 0 correct out of 115, percent 0.00
lf 10 category 1: 5 correct out of 115, percent 0.04
lf 11 category 1: 0 correct out of 115, percent 0.00
lf 12 category 1: 2 correct out of 115, percent 0.02
lf 13 category 1: 0 correct out of 115, percent 0.00
lf 14 category 1: 0 correct out of 115, percent 0.00


In [12]:
for k in range(L_test_neg.shape[1]):
    correct = np.sum([1 if L_test_neg[n, k] == 1 and y_test.to_numpy()[n] == 1 else 0 for n in range(L_test_neg.shape[0])])
    total = np.sum(test.Category == 1)
    print('lf %d category 1: %d correct out of %d, percent %.2f' % (k, correct, total, correct/total))

lf 0 category 1: 0 correct out of 14, percent 0.00
lf 1 category 1: 2 correct out of 14, percent 0.14
lf 2 category 1: 1 correct out of 14, percent 0.07
lf 3 category 1: 3 correct out of 14, percent 0.21
lf 4 category 1: 5 correct out of 14, percent 0.36
lf 5 category 1: 2 correct out of 14, percent 0.14
lf 6 category 1: 0 correct out of 14, percent 0.00
lf 7 category 1: 0 correct out of 14, percent 0.00
lf 8 category 1: 1 correct out of 14, percent 0.07
lf 9 category 1: 0 correct out of 14, percent 0.00
lf 10 category 1: 0 correct out of 14, percent 0.00
lf 11 category 1: 0 correct out of 14, percent 0.00
lf 12 category 1: 0 correct out of 14, percent 0.00
lf 13 category 1: 0 correct out of 14, percent 0.00
lf 14 category 1: 0 correct out of 14, percent 0.00


## BERT labels

In [13]:
bert_trains_probs = []
bert_tests_probs = []
for k in range(n_folds):
    with open('../output/bert-9f-multi/train/proba_%d.npy' % k, 'rb') as f:
        bert_trains_probs.append(np.load(f))
    with open('../output/bert-9f-multi/test/proba_%d.npy' % k, 'rb') as f:
        bert_tests_probs.append(np.load(f))
bert_trains = np.array([[1 if np.argmax(i) == 0 else 0 for i in fold] for fold in bert_trains_probs]).T
bert_tests = np.array([[1 if np.argmax(i) == 0 else 0 for i in fold] for fold in bert_tests_probs]).T
print(bert_trains.shape, bert_tests.shape)

(12326, 9) (1370, 9)


In [14]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    tmp_trains = bert_trains.copy()
    tmp_trains[tmp_trains == -1] = 0
    print(LFAnalysis(L=tmp_trains).lf_summary(np.array([1 if i == 1 else 0 for i in train_all.Category])))
    print(LFAnalysis(L=bert_trains).lf_summary(y_train.to_numpy()))

  Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  Emp. Acc.
0   [0, 1]       1.0       1.0   0.061577    12244         82   0.993347
1   [0, 1]       1.0       1.0   0.061577    12091        235   0.980935
2   [0, 1]       1.0       1.0   0.061577    12121        205   0.983368
3   [0, 1]       1.0       1.0   0.061577    12181        145   0.988236
4   [0, 1]       1.0       1.0   0.061577    12215        111   0.990995
5   [0, 1]       1.0       1.0   0.061577    12220        106   0.991400
6   [0, 1]       1.0       1.0   0.061577    12071        255   0.979312
7   [0, 1]       1.0       1.0   0.061577    12211        115   0.990670
8   [0, 1]       1.0       1.0   0.061577    12231         95   0.992293
  Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  Emp. Acc.
0   [0, 1]       1.0       1.0   0.061577       48         67   0.003894
1   [0, 1]       1.0       1.0   0.061577       63         52   0.005111
2   [0, 1]       1.0       1.0   0.061577       55 

In [15]:
for k in range(bert_trains.shape[1]):
    correct = np.sum([1 if bert_trains[n, k] == 1 and y_train.to_numpy()[n] == 1 else 0 for n in range(bert_trains.shape[0])])
    total = np.sum(train_all.Category == 1)
    print('fold %d category 1: %d correct out of %d, percent %.2f' % (k, correct, total, correct/total))

fold 0 category 1: 48 correct out of 115, percent 0.42
fold 1 category 1: 63 correct out of 115, percent 0.55
fold 2 category 1: 55 correct out of 115, percent 0.48
fold 3 category 1: 66 correct out of 115, percent 0.57
fold 4 category 1: 13 correct out of 115, percent 0.11
fold 5 category 1: 62 correct out of 115, percent 0.54
fold 6 category 1: 91 correct out of 115, percent 0.79
fold 7 category 1: 40 correct out of 115, percent 0.35
fold 8 category 1: 87 correct out of 115, percent 0.76


In [16]:
for k in range(bert_tests.shape[1]):
    correct = np.sum([1 if bert_tests[n, k] == 1 and y_train.to_numpy()[n] == 1 else 0 for n in range(bert_tests.shape[0])])
    total = np.sum(test.Category == 1)
    print('fold %d category 1: %d correct out of %d, percent %.2f' % (k, correct, total, correct/total))

fold 0 category 1: 2 correct out of 14, percent 0.14
fold 1 category 1: 8 correct out of 14, percent 0.57
fold 2 category 1: 7 correct out of 14, percent 0.50
fold 3 category 1: 3 correct out of 14, percent 0.21
fold 4 category 1: 0 correct out of 14, percent 0.00
fold 5 category 1: 4 correct out of 14, percent 0.29
fold 6 category 1: 9 correct out of 14, percent 0.64
fold 7 category 1: 3 correct out of 14, percent 0.21
fold 8 category 1: 9 correct out of 14, percent 0.64


## RF labels

In [17]:
rf_trains_probs = []
rf_tests_probs = []
for k in range(n_folds):
    with open('../output/rf-%df-multi/train/k%d.npy' % (n_folds, k), 'rb') as f:
        rf_trains_probs.append(np.load(f))
    with open('../output/rf-%df-multi/test/k%d.npy' % (n_folds, k), 'rb') as f:
        rf_tests_probs.append(np.load(f))
rf_trains = np.array([[1 if np.argmax(i) == 0 else -1 for i in fold] for fold in rf_trains_probs]).T
rf_tests = np.array([[1 if np.argmax(i) == 0 else -1 for i in fold] for fold in rf_tests_probs]).T
print(rf_trains.shape, rf_tests.shape)

(12326, 9) (1370, 9)


In [18]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    tmp_trains = rf_trains.copy()
    tmp_trains[tmp_trains == -1] = 0
    print(LFAnalysis(L=tmp_trains).lf_summary(np.array([1 if i == 1 else 0 for i in train_all.Category])))
    print(LFAnalysis(L=rf_trains).lf_summary(y_train.to_numpy()))

  Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  Emp. Acc.
0   [0, 1]       1.0       1.0   0.009086    12314         12   0.999026
1   [0, 1]       1.0       1.0   0.009086    12313         13   0.998945
2   [0, 1]       1.0       1.0   0.009086    12313         13   0.998945
3   [0, 1]       1.0       1.0   0.009086    12313         13   0.998945
4   [0, 1]       1.0       1.0   0.009086    12314         12   0.999026
5   [0, 1]       1.0       1.0   0.009086    12313         13   0.998945
6   [0, 1]       1.0       1.0   0.009086    12313         13   0.998945
7   [0, 1]       1.0       1.0   0.009086    12314         12   0.999026
8   [0, 1]       1.0       1.0   0.009086    12315         11   0.999108
  Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  Emp. Acc.
0      [1]  0.008356  0.008356        0.0      103          0        1.0
1      [1]  0.008275  0.008275        0.0      102          0        1.0
2      [1]  0.008275  0.008275        0.0      102 

In [19]:
for k in range(rf_trains.shape[1]):
    correct = np.sum([1 if rf_trains[n, k] == 1 and y_train.to_numpy()[n] == 1 else 0 for n in range(rf_trains.shape[0])])
    total = np.sum(train_all.Category == 1)
    print('fold %d category 1: %d correct out of %d, percent %.2f' % (k, correct, total, correct/total))

fold 0 category 1: 103 correct out of 115, percent 0.90
fold 1 category 1: 102 correct out of 115, percent 0.89
fold 2 category 1: 102 correct out of 115, percent 0.89
fold 3 category 1: 102 correct out of 115, percent 0.89
fold 4 category 1: 103 correct out of 115, percent 0.90
fold 5 category 1: 102 correct out of 115, percent 0.89
fold 6 category 1: 102 correct out of 115, percent 0.89
fold 7 category 1: 103 correct out of 115, percent 0.90
fold 8 category 1: 104 correct out of 115, percent 0.90


In [20]:
for k in range(rf_tests.shape[1]):
    correct = np.sum([1 if rf_tests[n, k] == 1 and y_test.to_numpy()[n] == 1 else 0 for n in range(rf_tests.shape[0])])
    total = np.sum(test.Category == 1)
    print('fold %d category 1: %d correct out of %d, percent %.2f' % (k, correct, total, correct/total))

fold 0 category 1: 0 correct out of 14, percent 0.00
fold 1 category 1: 0 correct out of 14, percent 0.00
fold 2 category 1: 0 correct out of 14, percent 0.00
fold 3 category 1: 0 correct out of 14, percent 0.00
fold 4 category 1: 3 correct out of 14, percent 0.21
fold 5 category 1: 0 correct out of 14, percent 0.00
fold 6 category 1: 1 correct out of 14, percent 0.07
fold 7 category 1: 1 correct out of 14, percent 0.07
fold 8 category 1: 0 correct out of 14, percent 0.00


## Simple Average

### RFs

In [21]:
recall(np.mean(rf_tests_probs, axis=0), test.Category, text='Simple Average RFs');

Simple Average RFs r 0.000 r0 1.000 rall 0.990 ravg 0.500


### BERTs

In [22]:
recall(np.mean(bert_tests_probs, axis=0), test.Category, text='Simple Average BERTs');

Simple Average BERTs r 0.000 r0 0.996 rall 0.986 ravg 0.498


## Snorkel

In [23]:
def snork(L_train, L_test, y_true):
    logger.setLevel(logging.CRITICAL)
    label_model = LabelModel(cardinality=2, verbose=True)
    label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

    label_model_acc = label_model.score(L=L_test, Y=y_true, tie_break_policy="random")["accuracy"]
    print(f"{'Label Accuracy:':<25} {label_model_acc * 100:.1f}%")
    label_prob = label_model.predict_proba(L=L_test)
    
    y = y_true[0].tolist()
    p = np.sum([1 for i in y if i == 1])
    n = len(y) - p
    
    tp = np.sum([1 for i in range(y_true.shape[0]) if label_prob[i][1] > 0.5 and y[i] == 1])
    fp = np.sum([1 for i in range(y_true.shape[0]) if label_prob[i][1] > 0.5 and y[i] != 1])
    tn = np.sum([1 for i in range(y_true.shape[0]) if label_prob[i][1] <= 0.5 and y[i] != 1])
    fn = np.sum([1 for i in range(y_true.shape[0]) if label_prob[i][1] <= 0.5 and y[i] == 1])
    
    print('tp %d fp %d tn %d fn %d' % (tp, fp, tn, fn))
    print('tpr %.3f fnr %.3f' % (tp / p, fn / p))
    print('fpr %.3f tnr %.3f' % (fp / n, tn / n))

### LFs only

In [24]:
snork(L_train, L_test, y_test);

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|                                                                             | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.277]
INFO:root:[100 epochs]: TRAIN:[loss=0.002]
 32%|████████████████████▋                                            | 159/500 [00:00<00:00, 1581.42epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.000]
INFO:root:[300 epochs]: TRAIN:[loss=0.000]
100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1698.09epoch/s]
INFO:root:Finished Training


Label Accuracy:           21.4%
tp 2 fp 513 tn 843 fn 12
tpr 0.143 fnr 0.857
fpr 0.378 tnr 0.622


In [25]:
snork(L_train_neg, L_test_neg, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1740.65epoch/s]


Label Accuracy:           64.3%
tp 8 fp 575 tn 781 fn 6
tpr 0.571 fnr 0.429
fpr 0.424 tnr 0.576


In [26]:
snork(L_train_abs, L_test_abs, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1739.18epoch/s]

Label Accuracy:           35.7%
tp 5 fp 1074 tn 282 fn 9
tpr 0.357 fnr 0.643
fpr 0.792 tnr 0.208


### RFs only

In [27]:
snork(rf_trains, rf_tests, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1835.28epoch/s]

Label Accuracy:           50.0%
tp 3 fp 0 tn 1356 fn 11
tpr 0.214 fnr 0.786
fpr 0.000 tnr 1.000


### BERT only

In [28]:
snork(bert_trains, bert_tests, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1838.95epoch/s]

Label Accuracy:           100.0%
tp 14 fp 75 tn 1281 fn 0
tpr 1.000 fnr 0.000
fpr 0.055 tnr 0.945


### LFs + RFs

In [29]:
train_stack = np.hstack((L_train, rf_trains))
test_stack = np.hstack((L_test, rf_tests))
snork(train_stack, test_stack, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1695.31epoch/s]


Label Accuracy:           21.4%
tp 2 fp 513 tn 843 fn 12
tpr 0.143 fnr 0.857
fpr 0.378 tnr 0.622


### LFs + BERT

In [30]:
train_stack = np.hstack((L_train, bert_trains))
test_stack = np.hstack((L_test, bert_tests))
snork(train_stack, test_stack, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1630.81epoch/s]

Label Accuracy:           14.3%
tp 2 fp 512 tn 844 fn 12
tpr 0.143 fnr 0.857
fpr 0.378 tnr 0.622


### RFs + BERT

In [31]:
train_stack = np.hstack((rf_trains, bert_trains))
test_stack = np.hstack((rf_tests, bert_tests))
snork(train_stack, test_stack, y_test);

100%|█████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1693.86epoch/s]


Label Accuracy:           100.0%
tp 14 fp 75 tn 1281 fn 0
tpr 1.000 fnr 0.000
fpr 0.055 tnr 0.945


### LFs + RFs + BERT

In [32]:
train_stack = np.hstack((L_train, rf_trains, bert_trains))
test_stack = np.hstack((L_test, rf_tests, bert_tests))
snork(train_stack, test_stack, y_test);

100%|██████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 849.01epoch/s]

Label Accuracy:           14.3%
tp 2 fp 512 tn 844 fn 12
tpr 0.143 fnr 0.857
fpr 0.378 tnr 0.622
